In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [4]:
# -------------------------------
# 1) Load prices
# -------------------------------
xlsx_path = Path("jse_prices_adjclose_new.xlsx")
if not xlsx_path.exists():
    raise FileNotFoundError(f"File not found: {xlsx_path}")

xls = pd.ExcelFile(xlsx_path)
sheet_name = xls.sheet_names[0]
px = pd.read_excel(xlsx_path, sheet_name=sheet_name)

# Heuristic to find date column
date_col = None
for c in px.columns:
    if str(c).strip().lower() in {"date", "timestamp", "time", "pricedate", "trade_date"}:
        date_col = c
        break
if date_col is None:
    for c in px.columns:
        if pd.api.types.is_datetime64_any_dtype(px[c]):
            date_col = c
            break
if date_col is None:
    date_col = px.columns[0]

px[date_col] = pd.to_datetime(px[date_col], errors="coerce")
px = px.dropna(subset=[date_col]).set_index(date_col).sort_index()
px = px.select_dtypes(include=[np.number])  # numeric columns = assets/indices
if px.shape[1] < 3:
    raise ValueError("Need at least 3 numeric columns (assets).")

In [8]:
# -------------------------------
# 2) Monthly returns from 1998-01-01
# -------------------------------
px = px.loc[px.index >= "2000-01-04"]
pm = px.resample("M").last()
retn = pm.pct_change().dropna(how="all")
retn = retn.dropna(axis=1, how="any")

C:\Users\Mphep\AppData\Local\Temp\ipykernel_26860\3450108467.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  pm = px.resample("M").last()


In [10]:
# -------------------------------
# 3) Market proxy & betas
# -------------------------------
def pick_market_column(columns):
    candidates = {
        "j203","j200","top40","jse top40","jse_top40","ftse/jse top 40",
        "alsi","jse alsi","jse_all_share","jse all share",
        "market","index","benchmark"
    }
    low = {c: str(c).lower() for c in columns}
    for c in columns:
        if low[c] in candidates:
            return c
    return None

mkt_col = pick_market_column(retn.columns)
if mkt_col is None:
    mkt = retn.mean(axis=1)         # equal-weighted market proxy
    assets = list(retn.columns)
else:
    mkt = retn[mkt_col]
    assets = [c for c in retn.columns if c != mkt_col]

R = retn[assets].copy()
M = mkt.reindex(R.index).fillna(0.0)

# CAPM betas: beta_i = Cov(r_i, r_m)/Var(r_m)
var_m = float(np.var(M.values, ddof=1))
betas = {}
for a in assets:
    cov_im = np.cov(R[a].values, M.values, ddof=1)[0, 1]
    betas[a] = (cov_im / var_m) if var_m > 0 else 0.0
betas = pd.Series(betas, index=assets)

In [30]:
betas.values

array([1.58217521, 0.98878675, 1.02189394, 0.81923558, 1.13059256,
       0.58486578, 0.87745891, 1.47774185, 0.93205254, 0.9595853 ,
       1.3950455 , 0.55411063, 0.95885176, 0.95367711, 0.76392658])

In [12]:
# -------------------------------
# 4) Mean vector and covariance
# -------------------------------
mu = R.mean()     # mean monthly returns
Sigma = R.cov()
R_target = float(np.median(mu.values))  # Exercise 8.7 target

In [26]:
R_target

0.013414467121277234

In [14]:
# -------------------------------
# 5) Optional cap-group constraints
# -------------------------------
'''
cap_groups_path = Path("/mnt/data/cap_groups.csv")
cap_groups = None
if cap_groups_path.exists():
    cg = pd.read_csv(cap_groups_path)
    cg["name"] = cg["name"].astype(str)
    cg["group"] = cg["group"].astype(str).str.upper().str[0]
    cg = cg[cg["name"].isin(assets)]
    cap_groups = {
        "L": [a for a in assets if a in set(cg.loc[cg.group=="L", "name"])],
        "M": [a for a in assets if a in set(cg.loc[cg.group=="M", "name"])],
        "S": [a for a in assets if a in set(cg.loc[cg.group=="S", "name"])],
    }
    cap_groups = {k: v for k, v in cap_groups.items() if len(v) > 0}
'''

'\ncap_groups_path = Path("/mnt/data/cap_groups.csv")\ncap_groups = None\nif cap_groups_path.exists():\n    cg = pd.read_csv(cap_groups_path)\n    cg["name"] = cg["name"].astype(str)\n    cg["group"] = cg["group"].astype(str).str.upper().str[0]\n    cg = cg[cg["name"].isin(assets)]\n    cap_groups = {\n        "L": [a for a in assets if a in set(cg.loc[cg.group=="L", "name"])],\n        "M": [a for a in assets if a in set(cg.loc[cg.group=="M", "name"])],\n        "S": [a for a in assets if a in set(cg.loc[cg.group=="S", "name"])],\n    }\n    cap_groups = {k: v for k, v in cap_groups.items() if len(v) > 0}\n'

In [16]:
# -------------------------------
# 6) Optimizer (SLSQP)
# -------------------------------
BETA_BAND = (0.9, 1.1)

def solve_portfolio(mu_vec, Sigma_mat, Rmin, betas_vec, beta_band=BETA_BAND, cap_groups=None):
    names = list(mu_vec.index)
    n = len(names)
    mu_np = mu_vec.values
    Sigma_np = Sigma_mat.loc[names, names].values
    beta_np = betas_vec.loc[names].values

    def obj(x):
        return 0.5 * float(x @ Sigma_np @ x)

    cons = []
    # Budget
    cons.append({"type": "eq", "fun": lambda x: np.sum(x) - 1.0})
    # Return floor
    cons.append({"type": "ineq", "fun": lambda x, mu_np=mu_np, Rmin=Rmin: float(mu_np @ x) - Rmin})
    # Beta band
    cons.append({"type": "ineq", "fun": lambda x, b=beta_np: float(b @ x) - beta_band[0]})
    cons.append({"type": "ineq", "fun": lambda x, b=beta_np: beta_band[1] - float(b @ x)})
    # Cap constraints (Exercise 8.6)
    if cap_groups and "L" in cap_groups and "M" in cap_groups and cap_groups["L"] and cap_groups["M"]:
        L_idx = [names.index(a) for a in cap_groups["L"] if a in names]
        M_idx = [names.index(a) for a in cap_groups["M"] if a in names]
        cons.append({"type": "eq", "fun": lambda x, L=L_idx, M=M_idx: np.sum(x[L]) - np.sum(x[M])})
    if cap_groups and "S" in cap_groups and "L" in cap_groups and cap_groups["S"] and cap_groups["L"]:
        L_idx = [names.index(a) for a in cap_groups["L"] if a in names]
        S_idx = [names.index(a) for a in cap_groups["S"] if a in names]
        cons.append({"type": "ineq", "fun": lambda x, L=L_idx, S=S_idx: np.sum(x[S]) - 2.0*np.sum(x[L])})
        cons.append({"type": "ineq", "fun": lambda x, L=L_idx, S=S_idx: 3.0*np.sum(x[L]) - np.sum(x[S])})

    # Long-only bounds
    bnds = [(0.0, 1.0)] * n
    x0 = np.full(n, 1.0 / n)

    res = minimize(obj, x0, method="SLSQP", bounds=bnds, constraints=cons,
                   options={"maxiter": 1000, "ftol": 1e-12})
    if not res.success:
        raise RuntimeError(f"Optimization failed: {res.message}")

    x = pd.Series(res.x, index=names)
    port = {
        "ExpRet": float(mu_np @ res.x),
        "Stdev": float(np.sqrt(res.x @ Sigma_np @ res.x)),
        "Beta": float(beta_np @ res.x),
    }
    return x, port

In [20]:
# -------------------------------
# 7) Exercise 8.7 — baseline + 4 perturbations + average
# -------------------------------
x0, p0 = solve_portfolio(mu, Sigma, R_target, betas, beta_band=BETA_BAND, cap_groups=None)

rng = np.random.default_rng(42)
x_list = [x0]
p_list = [p0]
for _ in range(4):
    lo, hi = 0.95 * mu.values, 1.05 * mu.values
    mu_tilde = pd.Series(rng.uniform(lo, hi), index=mu.index)
    xk, pk = solve_portfolio(mu_tilde, Sigma, R_target, betas, beta_band=BETA_BAND, cap_groups=None)
    x_list.append(xk); p_list.append(pk)

x_avg = pd.concat(x_list, axis=1).mean(axis=1).rename("Average")

# Metrics for the averaged portfolio
Sigma_np = Sigma.loc[x_avg.index, x_avg.index].values
mu_np = mu.loc[x_avg.index].values
beta_np = betas.loc[x_avg.index].values
x_avg_np = x_avg.values
p_avg = {
    "ExpRet": float(mu_np @ x_avg_np),
    "Stdev": float(np.sqrt(x_avg_np @ Sigma_np @ x_avg_np)),
    "Beta": float(beta_np @ x_avg_np),
}


In [38]:
x0.values

array([0.03040821, 0.03978716, 0.00732206, 0.078593  , 0.09315509,
       0.12113741, 0.09855994, 0.04123555, 0.01659101, 0.06985211,
       0.05727802, 0.14825659, 0.04761264, 0.06397557, 0.08623564])

In [36]:
p0

{'ExpRet': 0.013414467121276887,
 'Stdev': 0.04812391975005027,
 'Beta': 0.8999999999999577}

In [24]:
p_avg

{'ExpRet': 0.013382185206111855,
 'Stdev': 0.048119910809988675,
 'Beta': 0.9000000000000274}

In [22]:
# -------------------------------
# 8) Save outputs
# -------------------------------
out_dir = Path("part2_outputs_new")
out_dir.mkdir(parents=True, exist_ok=True)

weights_df = pd.concat(
    [
        x_list[0].rename("Baseline"),
        x_list[1].rename("Perturb_1"),
        x_list[2].rename("Perturb_2"),
        x_list[3].rename("Perturb_3"),
        x_list[4].rename("Perturb_4"),
        x_avg
    ],
    axis=1
)
summary_rows = []
for name, p in zip(["Baseline","Perturb_1","Perturb_2","Perturb_3","Perturb_4"], p_list):
    summary_rows.append({"Portfolio": name, **p})
summary_rows.append({"Portfolio": "Average", **p_avg})
summary_df = pd.DataFrame(summary_rows).set_index("Portfolio")

weights_df.to_csv(out_dir / "exercise_8_7_weights.csv")
summary_df.to_csv(out_dir / "exercise_8_7_summary.csv")
betas.sort_index().to_csv(out_dir / "asset_betas.csv", header=["beta"])

print("Saved to:", out_dir)

Saved to: part2_outputs_new


In [52]:
summary_df

,ExpRet,Stdev,Beta
Portfolio,,,
Baseline,0.013414,0.048124,0.9
Perturb_1,0.013414,0.048109,0.9
Perturb_2,0.013414,0.048113,0.9
Perturb_3,0.013414,0.048118,0.9
Perturb_4,0.013414,0.048154,0.9
Average,0.013382,0.048120,0.9


In [50]:
weights_df

,Baseline,Perturb_1,Perturb_2,Perturb_3,Perturb_4,Average
Harmony Gold Mining Company Ltd,0.030408,0.029967,0.029840,0.030483,0.031607,0.030461
MTN Group Ltd,0.039787,0.041909,0.041147,0.040912,0.037693,0.040290
Mr Price Group Ltd,0.007322,0.004208,0.004753,0.005832,0.009473,0.006318
Aspen Pharmacare Holdings Ltd,0.078593,0.078558,0.078735,0.078511,0.079528,0.078785
Gold Fields Ltd,0.093155,0.092478,0.093050,0.092777,0.092288,0.092749
Shoprite Holdings Ltd,0.121137,0.118492,0.119498,0.119536,0.123574,0.120447
Standard Bank Group Ltd,0.098560,0.099418,0.097702,0.097471,0.093904,0.097411
Impala Platinum Holdings Ltd,0.041236,0.040482,0.040967,0.040976,0.042454,0.041223
FirstRand Ltd,0.016591,0.010355,0.014078,0.014372,0.024203,0.015920
Absa Group Ltd,0.069852,0.071562,0.071668,0.071588,0.070430,0.071020
